In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from sklearn.utils import shuffle

In [9]:
dateparse = lambda x: datetime.strptime(x, '%d-%m-%Y')
df = pd.read_csv("Challenge_2_submission_template.csv", parse_dates=['Date of Harvest'], date_parser=dateparse)
df["Predicted Rice Yield (kg/ha)"] = df.index
df

,ID No,District,Latitude,Longitude,"Season(SA = Summer Autumn, WS = Winter Spring)","Rice Crop Intensity(D=Double, T=Triple)",Date of Harvest,Field size (ha),Predicted Rice Yield (kg/ha)
0,1,Chau_Phu,10.542192,105.187920,WS,T,2022-04-10,1.40,0
1,2,Chau_Thanh,10.400189,105.331053,SA,T,2022-07-15,1.32,1
2,3,Chau_Phu,10.505489,105.203926,SA,D,2022-07-14,1.40,2
3,4,Chau_Phu,10.523520,105.138274,WS,D,2022-04-10,1.80,3
4,5,Thoai_Son,10.294660,105.248528,SA,T,2022-07-20,2.20,4
...,...,...,...,...,...,...,...,...,...
95,96,Chau_Thanh,10.435839,105.132981,SA,D,2022-07-26,1.21,95
96,97,Chau_Phu,10.529357,105.147388,WS,T,2022-04-10,2.00,96
97,98,Chau_Thanh,10.452537,105.205118,SA,T,2022-07-20,5.50,97
98,99,Chau_Thanh,10.394341,105.126836,SA,T,2022-07-14,4.40,98


In [17]:
# Create a function that will mask pixels with a given type
bit_flags = {
            'dilated_cloud': 1<<1,
            'cloud': 1<<3,
            'shadow': 1<<4, 
}
def get_mask(mask, flags_list):
    # Create the result mask filled with zeros and the same shape as the mask
    final_mask = np.zeros_like(mask)
    # Loop through the flags  
    for flag in flags_list:
        # get the mask for each flag
        flag_mask = np.bitwise_and(mask, bit_flags[flag])
        # add it to the final flag
        final_mask = final_mask | flag_mask
    return final_mask > 0

def get_answer(answers, cloud, shadow):
    def get_avg(x):
        x_sum = x.sum(axis=1)
        return np.count_nonzero(x_sum), (x_sum*np.arange(len(test))).sum()/x_sum.sum()
    cloud_nb, cloud_avg = get_avg(cloud)
    shadow_nb, shadow_avg = get_avg(shadow)
    if cloud_nb > 10 and shadow_nb > 10:
        if cloud_avg > shadow_avg:
            answers["WS"] += 1
        else:
            answers["SA"] += 1
            
    

def conditions(x):
    indice = x["Predicted Rice Yield (kg/ha)"]
    with open(f"inputs_lstm/{indice}.npy", "rb") as f:
        arr = np.load(f)[:, 4]
    answers = {"WS":0, "SA":0}
    for sample in arr:
        cloud = get_mask(sample,
                   ['cloud', 'dilated_cloud'])
        shadow = get_mask(sample,
                   ['shadow'])
        get_answer(answers, cloud, shadow)
    if answers["WS"] > answers["SA"]:
        return "WS"
    else:
        return "SA"

In [21]:
x = df.iloc[0]

In [22]:
indice = x["Predicted Rice Yield (kg/ha)"]
with open(f"inputs_lstm/{indice}.npy", "rb") as f:
    arr = np.load(f)[:, 4]
answers = {"WS":0, "SA":0}
for sample in arr:
    cloud = get_mask(sample,
               ['cloud', 'dilated_cloud'])
    shadow = get_mask(sample,
               ['shadow'])
    get_answer(answers, cloud, shadow)
if answers["WS"] > answers["SA"]:
    return "WS"
else:
    return "SA"

TypeError: ufunc 'bitwise_and' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [23]:
sample

array([[21824., 21824., 21824., ..., 21824., 21824., 21824.],
       [21824., 21824., 21824., ..., 21824., 21824., 21824.],
       [21824., 21824., 21824., ..., 21824., 21824., 21824.],
       ...,
       [21824., 21824., 21824., ..., 22080., 22080., 22080.],
       [21824., 21824., 21824., ..., 22080., 22080., 22080.],
       [21824., 21824., 21824., ..., 22080., 22080., 22080.]])

In [18]:
df["test"] = df.apply(conditions, axis=1)
df

TypeError: ufunc 'bitwise_and' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''